In [32]:
import numpy as np
import os
import matplotlib.pyplot as plt
import h5py
from scipy import interpolate
from ccfj import GetStationPairs
import pandas as pd
import yaml
import geopandas as gp
import openpyxl
from openpyxl import load_workbook
from concurrent.futures import ThreadPoolExecutor

from scipy.interpolate import make_interp_spline
from scipy.stats import ks_2samp
from scipy.spatial import ConvexHull

In [33]:
import sys
sys.path.append(r'../tools_F-J/')
from toollib_standard import maplib
from toollib_standard import mathlib
from toollib_standard import filelib
from toollib_standard import stacklib
from toollib_standard import plotlib

In [34]:
with open('a-project_repar.yml', 'r', encoding='utf-8') as f:
    proj = yaml.load(f.read(), Loader=yaml.FullLoader)
name_project = proj['name']
#name_project = 'project_repartrition/output_repar_03-01/'               # Harmon server
name_project

'project_repartition_v3.0/output_repar_01-01/'

In [35]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

dir_CC_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project:  /shdisk/rem2/Harmon/F-J/San/project_repartition_v3.0/output_repar_01-01/


In [36]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [37]:
name_project_probes = info_basic['name_project_probes']
name_project_targets = info_basic['name_project_targets']
dir_project_probes = os.path.join(dir_project_workspace, name_project_probes)
dir_project_targets = os.path.join(dir_project_workspace, name_project_targets)
print('dir_project_probes: ', dir_project_probes)
print('dir_project_targets: ', dir_project_targets)

dir_project_probes:  /shdisk/rem2/Harmon/F-J/San/project/output_FJSJ_17-01/
dir_project_targets:  /shdisk/rem2/Harmon/F-J/San/project/output_FJSJ_17-02/


In [38]:
stalistname_all = info_basic['stalistname_all']
stainfo = pd.read_excel(stalistname_all)
nsta_all = len(stainfo.iloc[:,0])
StationPairs_all = GetStationPairs(nsta_all)
nPairs_all = int(len(StationPairs_all)/2)
stalist_all = stainfo['Station'].tolist()
lat_all = stainfo['latitude'].tolist() 
lon_all = stainfo['longitude'].tolist()

In [39]:
stalistname = dir_project+ info_basic['stalistname']
stainfo_all = pd.read_excel(stalistname,sheet_name=None)

In [40]:
stalistname = dir_project + 'subwork_location.xlsx'
loc_key = pd.read_excel(stalistname,sheet_name='location')
lat_key = loc_key['lat_centroid'].tolist()
lon_key = loc_key['lon_centroid'].tolist()

In [41]:
dir_image = dir_project+info_basic['dir_image']
print('dir_image: ', dir_image)

dir_image:  /shdisk/rem2/Harmon/F-J/San/project_repartition_v3.0/output_repar_01-01/image_01-01/


# Be careful to delete!

In [42]:
nums_delete = [30,247,293,319,353,354,364,365,398,399,428,429,430,431,444,445,448,452,457,459,469,478,479,487,488,509,520,521,523,524,525,568,569,570,572,594,639,673,675]
#nums_delete = [29,106,115,383,426,432,444,469,458,506,527,531,549,550,551,552,553,574,583,596,623,641,671,672]

In [43]:
key_subworks = []
for num in nums_delete:
    tag = str(num)
    print(tag)
    key_subwork0 = [key_subwork for key_subwork in info_basic['key_subworks'] if tag == key_subwork[-len(str(tag)):]]
    if key_subwork0 != []:
        key_subworks.append(key_subwork0[0])
key_subworks

30
247
293
319
353
354
364
365
398
399
428
429
430
431
444
445
448
452
457
459
469
478
479
487
488
509
520
521
523
524
525
568
569
570
572
594
639
673
675


['41-03--30',
 '07-09--247',
 '07-10--293',
 '38-10--319',
 '28-11--353',
 '29-11--354',
 '41-11--364',
 '42-11--365',
 '27-12--398',
 '28-12--399',
 '11-13--428',
 '12-13--429',
 '13-13--430',
 '14-13--431',
 '28-13--445',
 '32-13--448',
 '36-13--452',
 '41-13--457',
 '43-13--459',
 '15-14--478',
 '16-14--479',
 '24-14--487',
 '25-14--488',
 '47-14--509',
 '12-15--520',
 '13-15--521',
 '15-15--523',
 '16-15--524',
 '17-15--525',
 '15-16--568',
 '16-16--569',
 '18-16--570',
 '21-16--572',
 '43-16--594',
 '44-17--639',
 '40-18--673',
 '44-18--675']

### Renew information

In [44]:
probe_accept_origin = info_basic_bi['probe_accept']
key_probe = info_basic_bi['probe']

In [45]:

for key_subwork in key_subworks:
    target = key_subwork.split('--')[0]
    if target in probe_accept_origin.keys():
        del probe_accept_origin[target]
    if target in key_probe.keys():
        del key_probe[key_subwork]
info_basic_bi['probe_accept'] = probe_accept_origin
info_basic_bi['probe'] = key_probe

In [46]:
filename = dir_project+'stations_info.xlsx'
wb = load_workbook(filename)
for key in key_subworks:
    #删除sheet
    print(key)
    try:
        
        wb.remove(wb[key])
        
    except:
        pass
wb.save(filename)

41-03--30
07-09--247
07-10--293
38-10--319
28-11--353
29-11--354
41-11--364
42-11--365
27-12--398
28-12--399
11-13--428
12-13--429
13-13--430
14-13--431
28-13--445
32-13--448
36-13--452
41-13--457
43-13--459
15-14--478
16-14--479
24-14--487
25-14--488
47-14--509
12-15--520
13-15--521
15-15--523
16-15--524
17-15--525
15-16--568
16-16--569
18-16--570
21-16--572
43-16--594
44-17--639
40-18--673
44-18--675


In [47]:
filename = dir_project + 'subwork_location.xlsx'
loc_info = pd.read_excel(filename,sheet_name='location')
lat_centroid_all_origin = loc_info['lat_centroid'].tolist()
lon_centroid_all_origin = loc_info['lon_centroid'].tolist()
stations = loc_info['key_subwork'].tolist()
for key in key_subworks:
    try:
        index = stations.index(key)
        # 删除对应的行
        loc_info.drop(index=index, inplace=True)
        # 保存
        loc_info.to_excel(filename, sheet_name='location', index=False)
    except:
        pass

In [48]:
for key in key_subworks:
    del info_basic['key_subworks'][info_basic['key_subworks'].index(key)]

In [49]:
with open(dir_project+'Basic_info.yml', 'w', encoding='utf-8') as f:
   yaml.dump(data=info_basic, stream=f, allow_unicode=True)
np.save(dir_project + 'Basic_info.npy', info_basic_bi)

### Delete stack and ds

In [50]:
dir_stack = dir_project + info_basic['dir_stack']
dir_ds = dir_project + info_basic['dir_ds']

In [51]:
list_stack = os.listdir(dir_stack)
list_ds = os.listdir(dir_ds)

In [52]:
for key in key_subworks:
    stack_this = key + '_gather_linear.h5'
    stack_timewindow = key + '_gather_timewindow.h5'
    ds_this = 'ds_'+key +'.h5'
    if stack_this in list_stack:
        print('delete ', stack_this)
        os.remove(dir_stack + stack_this)
    if stack_timewindow in list_stack:
        print('delete ', stack_timewindow)
        os.remove(dir_stack + stack_timewindow)
    if ds_this in list_ds:
        print('delete ', ds_this)
        os.remove(dir_ds + ds_this)